<a href="https://colab.research.google.com/github/xnileshtiwari/AI-Fusion/blob/main/Descript_like_wordhighlights_subtitles/Descript_like_wordhighlights_subtitles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code and files for this notebook: https://github.com/ramsrigouthamg/Supertranslate.ai/tree/main/Descript_like_wordhighlights_subtitles

Author: [Ramsri Goutham](https://twitter.com/ramsri_goutham)

In [1]:
!pip install --quiet ipython-autotime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.1 MB/s eta 0:00:00


In [2]:
%load_ext autotime

time: 233 µs (started: 2025-04-05 16:59:58 +00:00)


## Step1: Download video or upload the video in mp4 format that you want to convert to fancy word-level subtitles (audiogram)

In [3]:
import os

mp4videoURL = "https://github.com/ramsrigouthamg/Supertranslate.ai/raw/main/Descript_like_wordhighlights_subtitles/Intro.mp4"
videofilename = mp4videoURL.split('/')[-1]
print (videofilename)


Intro.mp4


In [4]:
import urllib.request
urllib.request.urlretrieve(mp4videoURL, videofilename)

('Intro.mp4', <http.client.HTTPMessage at 0x7d60d6e19710>)

## Step2: Extract audio (mp3) from the video file

In [5]:
!pip install ffmpeg-python==0.2.0

In [6]:
import ffmpeg

audiofilename = videofilename.replace(".mp4",'.mp3')

# Create the ffmpeg input stream
input_stream = ffmpeg.input(videofilename)

# Extract the audio stream from the input stream
audio = input_stream.audio

# Save the audio stream as an MP3 file
output_stream = ffmpeg.output(audio, audiofilename)

# Overwrite output file if it already exists
output_stream = ffmpeg.overwrite_output(output_stream)

ffmpeg.run(output_stream)

print (audiofilename)


Intro.mp3


In [7]:
from IPython.display import Audio

Audio(audiofilename)


time: 10.5 ms (started: 2025-04-05 17:00:35 +00:00)


## Step3: Get word-level transcription (speech-to-text) from audio using OpenAI's Whisper

In [7]:
!pip install git+https://github.com/openai/whisper.git@248b6cb124225dd263bb9bd32d060b6517e067f8 -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [8]:
# Downgrade NumPy to a version compatible with your PyTorch
!pip uninstall -y numpy
!pip install numpy==1.24.3  # A stable 1.x version

# Restart the runtime after this installation

Found existing installation: numpy 1.24.3
Uninstalling numpy-1.24.3:
  Successfully uninstalled numpy-1.24.3
  Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.21.0+cu124 requires torch==2.6.0, but you have torch 2.0.1 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.3 which is incompatible.
albucore 0.0.23 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
pymc 5.21.1 requires numpy>=1.25.0, but you have numpy 1.24.3 which 

In [2]:
audiofilename = "/content/Intro.mp3"

In [3]:
import whisper
                                          #might take some time (approx 3- 5min depending on audio length)
model = whisper.load_model("small")
result = model.transcribe(audiofilename,word_timestamps=True)
print(result)

{'text': ' Hello everyone, my name is Ramshree and welcome to my YouTube channel. In this YouTube channel I share my learnings from building three AIS as apps super meme.ai, question.ai and super translate.ai', 'segments': [{'id': 0, 'seek': 0, 'start': 0.28, 'end': 5.18, 'text': ' Hello everyone, my name is Ramshree and welcome to my YouTube channel. In this', 'tokens': [50364, 2425, 1518, 11, 452, 1315, 307, 9078, 2716, 701, 293, 2928, 281, 452, 3088, 2269, 13, 682, 341, 50620], 'temperature': 0.0, 'avg_logprob': -0.3415653155400203, 'compression_ratio': 1.3971631205673758, 'no_speech_prob': 0.0469832606613636, 'words': [{'word': ' Hello', 'start': 0.28, 'end': 0.88, 'probability': 0.7725649476051331}, {'word': ' everyone,', 'start': 0.88, 'end': 1.28, 'probability': 0.7741734385490417}, {'word': ' my', 'start': 1.62, 'end': 1.96, 'probability': 0.9555560946464539}, {'word': ' name', 'start': 1.96, 'end': 2.1, 'probability': 0.9984112977981567}, {'word': ' is', 'start': 2.1, 'end': 2

In [ ]:
# total text
print (result['text'])

In [ ]:
# each subtitle segment
for each in result['segments']:
  print (each)

In [4]:
wordlevel_info = []

for each in result['segments']:
  words = each['words']
  for word in words:
    # print (word['word'], "  ",word['start']," - ",word['end'])
    wordlevel_info.append({'word':word['word'].strip(),'start':word['start'],'end':word['end']})


In [5]:
wordlevel_info

[{'word': 'Hello', 'start': 0.28, 'end': 0.88},
 {'word': 'everyone,', 'start': 0.88, 'end': 1.28},
 {'word': 'my', 'start': 1.62, 'end': 1.96},
 {'word': 'name', 'start': 1.96, 'end': 2.1},
 {'word': 'is', 'start': 2.1, 'end': 2.22},
 {'word': 'Ramshree', 'start': 2.22, 'end': 2.74},
 {'word': 'and', 'start': 2.74, 'end': 3.0},
 {'word': 'welcome', 'start': 3.0, 'end': 3.28},
 {'word': 'to', 'start': 3.28, 'end': 3.5},
 {'word': 'my', 'start': 3.5, 'end': 3.66},
 {'word': 'YouTube', 'start': 3.66, 'end': 3.84},
 {'word': 'channel.', 'start': 3.84, 'end': 4.28},
 {'word': 'In', 'start': 4.82, 'end': 4.98},
 {'word': 'this', 'start': 4.98, 'end': 5.18},
 {'word': 'YouTube', 'start': 5.18, 'end': 5.42},
 {'word': 'channel', 'start': 5.42, 'end': 5.8},
 {'word': 'I', 'start': 5.8, 'end': 6.0},
 {'word': 'share', 'start': 6.0, 'end': 6.3},
 {'word': 'my', 'start': 6.3, 'end': 6.56},
 {'word': 'learnings', 'start': 6.56, 'end': 7.14},
 {'word': 'from', 'start': 7.14, 'end': 7.3},
 {'word': 

## Step4: Store word-level timestamps into JSON file

In [6]:
import json
with open('data.json', 'w') as f:
    json.dump(wordlevel_info, f,indent=4)

## Step5: Correct and save JSON file if there are transcription errors or if you need to merge any words into phrases. Load the new edited JSON file with new word-level timestamps

In [7]:
import json

with open('data.json', 'r') as f:
    wordlevel_info_modified = json.load(f)


In [ ]:
wordlevel_info_modified

## Step6: Convert word-level timestamps JSON to line-level timestamps JSON. We want to display one line at a time and highlight words in it as they are spoken so we do this transformation.

In [8]:
def split_text_into_lines(data):

    MaxChars = 80
    #maxduration in seconds
    MaxDuration = 3.0
    #Split if nothing is spoken (gap) for these many seconds
    MaxGap = 1.5

    subtitles = []
    line = []
    line_duration = 0
    line_chars = 0


    for idx,word_data in enumerate(data):
        word = word_data["word"]
        start = word_data["start"]
        end = word_data["end"]

        line.append(word_data)
        line_duration += end - start

        temp = " ".join(item["word"] for item in line)


        # Check if adding a new word exceeds the maximum character count or duration
        new_line_chars = len(temp)

        duration_exceeded = line_duration > MaxDuration
        chars_exceeded = new_line_chars > MaxChars
        if idx>0:
          gap = word_data['start'] - data[idx-1]['end']
          # print (word,start,end,gap)
          maxgap_exceeded = gap > MaxGap
        else:
          maxgap_exceeded = False


        if duration_exceeded or chars_exceeded or maxgap_exceeded:
            if line:
                subtitle_line = {
                    "word": " ".join(item["word"] for item in line),
                    "start": line[0]["start"],
                    "end": line[-1]["end"],
                    "textcontents": line
                }
                subtitles.append(subtitle_line)
                line = []
                line_duration = 0
                line_chars = 0


    if line:
        subtitle_line = {
            "word": " ".join(item["word"] for item in line),
            "start": line[0]["start"],
            "end": line[-1]["end"],
            "textcontents": line
        }
        subtitles.append(subtitle_line)

    return subtitles

In [9]:
linelevel_subtitles = split_text_into_lines(wordlevel_info_modified)
print (linelevel_subtitles)

[{'word': 'Hello everyone, my name is Ramshree and welcome to my', 'start': 0.28, 'end': 3.66, 'textcontents': [{'word': 'Hello', 'start': 0.28, 'end': 0.88}, {'word': 'everyone,', 'start': 0.88, 'end': 1.28}, {'word': 'my', 'start': 1.62, 'end': 1.96}, {'word': 'name', 'start': 1.96, 'end': 2.1}, {'word': 'is', 'start': 2.1, 'end': 2.22}, {'word': 'Ramshree', 'start': 2.22, 'end': 2.74}, {'word': 'and', 'start': 2.74, 'end': 3.0}, {'word': 'welcome', 'start': 3.0, 'end': 3.28}, {'word': 'to', 'start': 3.28, 'end': 3.5}, {'word': 'my', 'start': 3.5, 'end': 3.66}]}, {'word': 'YouTube channel. In this YouTube channel I share my learnings from', 'start': 3.66, 'end': 7.3, 'textcontents': [{'word': 'YouTube', 'start': 3.66, 'end': 3.84}, {'word': 'channel.', 'start': 3.84, 'end': 4.28}, {'word': 'In', 'start': 4.82, 'end': 4.98}, {'word': 'this', 'start': 4.98, 'end': 5.18}, {'word': 'YouTube', 'start': 5.18, 'end': 5.42}, {'word': 'channel', 'start': 5.42, 'end': 5.8}, {'word': 'I', 'star

In [ ]:
for line in linelevel_subtitles:
  json_str = json.dumps(line, indent=4)
  print(json_str)

## Step7: Use Moviepy to create an audiogram with word-level highlights as they are spoken

In [11]:
!pip install moviepy==2.0.0.dev2 -q
!pip install imageio==2.25.1 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.1/400.1 kB 23.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 71.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.25.2 requires imageio!=2.35.0,>=2.33, but you have imageio 2.25.1 which is incompatible.


In [12]:
!apt install imagemagick -q

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts imagemagick-6-common
  imagemagick-6.q16 libdjvulibre-text libdjvulibre21 libfftw3-double3 libgs9 libgs9-common libidn12
  libijs-0.35 libjbig2dec0 libjxr-tools libjxr0 liblqr-1-0 libmagickcore-6.q16-6
  libmagickcore-6.q16-6-extra libmagickwand-6.q16-6 libnetpbm10 libwmflite-0.2-7 netpbm
  poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre ghostscript-x imagemagick-doc
  autotrace cups-bsd | lpr | lprng enscript gimp gnuplot grads hp2xx html2ps libwmf-bin mplayer
  povray radiance sane-utils texlive-base-bin transfig ufraw-batch libfftw3-bin libfftw3-dev
  inkscape poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho fonts-japanese-gothic
  | fonts-ipafont-gothic fonts-arphic-ukai fonts-arphic-uming fonts-nanum
The followin

In [13]:
!cat /etc/ImageMagick-6/policy.xml | sed 's/none/read,write/g'> /etc/ImageMagick-6/policy.xml

In [14]:
from moviepy.editor import TextClip, CompositeVideoClip, ColorClip
import numpy as np


def create_caption(textJSON, framesize,font = "Helvetica-Bold",fontsize=80, color='white', bgcolor='blue'):
    wordcount = len(textJSON['textcontents'])
    full_duration = textJSON['end']-textJSON['start']

    word_clips = []
    xy_textclips_positions =[]

    x_pos = 0
    y_pos = 0
    # max_height = 0
    frame_width = framesize[0]
    frame_height = framesize[1]
    x_buffer = frame_width*1/10
    y_buffer = frame_height*1/5

    space_width = ""
    space_height = ""

    for index,wordJSON in enumerate(textJSON['textcontents']):
      duration = wordJSON['end']-wordJSON['start']
      word_clip = TextClip(wordJSON['word'], font = font,fontsize=fontsize, color=color).set_start(textJSON['start']).set_duration(full_duration)
      word_clip_space = TextClip(" ", font = font,fontsize=fontsize, color=color).set_start(textJSON['start']).set_duration(full_duration)
      word_width, word_height = word_clip.size
      space_width,space_height = word_clip_space.size
      if x_pos + word_width+ space_width > frame_width-2*x_buffer:
            # Move to the next line
            x_pos = 0
            y_pos = y_pos+ word_height+40

            # Store info of each word_clip created
            xy_textclips_positions.append({
                "x_pos":x_pos+x_buffer,
                "y_pos": y_pos+y_buffer,
                "width" : word_width,
                "height" : word_height,
                "word": wordJSON['word'],
                "start": wordJSON['start'],
                "end": wordJSON['end'],
                "duration": duration
            })

            word_clip = word_clip.set_position((x_pos+x_buffer, y_pos+y_buffer))
            word_clip_space = word_clip_space.set_position((x_pos+ word_width +x_buffer, y_pos+y_buffer))
            x_pos = word_width + space_width
      else:
            # Store info of each word_clip created
            xy_textclips_positions.append({
                "x_pos":x_pos+x_buffer,
                "y_pos": y_pos+y_buffer,
                "width" : word_width,
                "height" : word_height,
                "word": wordJSON['word'],
                "start": wordJSON['start'],
                "end": wordJSON['end'],
                "duration": duration
            })

            word_clip = word_clip.set_position((x_pos+x_buffer, y_pos+y_buffer))
            word_clip_space = word_clip_space.set_position((x_pos+ word_width+ x_buffer, y_pos+y_buffer))

            x_pos = x_pos + word_width+ space_width


      word_clips.append(word_clip)
      word_clips.append(word_clip_space)


    for highlight_word in xy_textclips_positions:

      word_clip_highlight = TextClip(highlight_word['word'], font = font,fontsize=fontsize, color=color,bg_color = bgcolor).set_start(highlight_word['start']).set_duration(highlight_word['duration'])
      word_clip_highlight = word_clip_highlight.set_position((highlight_word['x_pos'], highlight_word['y_pos']))
      word_clips.append(word_clip_highlight)

    return word_clips



In [16]:
videofilename = "/content/Intro.mp4"

In [17]:

from moviepy.editor import TextClip, CompositeVideoClip, concatenate_videoclips,VideoFileClip, ColorClip
frame_size = (1080,1080)

all_linelevel_splits=[]

for line in linelevel_subtitles:
  out = create_caption(line,frame_size)
  all_linelevel_splits.extend(out)


# Load the input video
input_video = VideoFileClip(videofilename)
# Get the duration of the input video
input_video_duration = input_video.duration
# Create a color clip with the given frame size, color, and duration
background_clip = ColorClip(size=frame_size, color=(0, 0, 0)).set_duration(input_video_duration)

# If you want to overlay this on the original video uncomment this and also change frame_size, font size and color accordingly.
# final_video = CompositeVideoClip([input_video] + all_linelevel_splits)

final_video = CompositeVideoClip([background_clip] + all_linelevel_splits)

# Set the audio of the final video to be the same as the input video
final_video = final_video.set_audio(input_video.audio)

# Save the final clip as a video file with the audio included
final_video.write_videofile("output.mp4", fps=24, codec="libx264", audio_codec="aac")



Moviepy - Building video output.mp4.
MoviePy - Writing audio in outputTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output.mp4



Moviepy - Done !
Moviepy - video ready output.mp4


## Step8: Visualize the final Descript like audiogram video

In [18]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open("output.mp4",'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)